# Trabalho Final

## Coleta Preparação e Análise de Dados

#### Aléxia de Jesus Dorneles Pereira, Arthur Land Avila, Gustavo Giongo Lottermann, Matheus Gonzaga Krebs e Vicente Hofmeister

Trabalho feito utilizando [biblioteca Pysus](https://github.com/AlertaDengue/PySUS/tree/main). Caso necessário, confira a [documentação](https://pysus.readthedocs.io/en/latest/).

## Pysus

In [85]:
import os
from IPython.display import display, Javascript
import pandas as pd
from pysus.online_data import SINAN as SINAN_Online
from pysus.online_data import SINASC as SINASC_Online
from pysus.online_data import SIM as SIM_ONLINE
from pysus.ftp.databases.sinan import SINAN
from pysus.ftp.databases.sinasc import SINASC
from pysus.ftp.databases.sim import SIM

Import de bibliotecas

A sessão de código a seguir salva em listas os anos que os dados de sinan, sinasc e sim estão disponiveis

In [86]:
states = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', \
          'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', \
          'RR', 'SC', 'SP', 'SE', 'TO']

sinan_years = SINAN_Online.get_available_years("DENG")
sinasc_years = SINASC_Online.get_available_years(group= 'DN', states = states)
sim_years = SIM_ONLINE.get_available_years(group= 'CID10', states = states)

Confere os anos em que todas bases de dados estão disponiveis simultaneamente

In [87]:
avaliable_years = []

for year in sinan_years:
      if year in sinasc_years and year in sim_years:
            avaliable_years.append(year)

#### SINAN

busca arquivos SINAN baseado nos anos definidos em years, descreve-os e realiza download dos datasets

In [88]:
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
sinan = SINAN().load()
sinan_files = sinan.get_files(dis_code=['DENG'], year=years)

parquet_path = os.getcwd() + "/Data/Parquet/"
csv_path = os.getcwd() + '/Data/CSV/'

if not 'Parquet' in os.listdir(os.getcwd() + "/Data") :
      os.mkdir(parquet_path)

if not 'CSV' in os.listdir(os.getcwd() + "/Data") :
      os.mkdir(csv_path)

for file in sinan_files:
      if not file.basename.replace('.dbc', '.parquet') in os.listdir(parquet_path) and not file.basename.replace('.dbc', '.csv') in os.listdir(csv_path):
            print(sinan.describe(file))
            sinan.download(file, local_dir=parquet_path)

{'name': 'DENGBR12.dbc', 'disease': 'Dengue', 'year': 2012, 'size': '31.3 MB', 'last_update': '2022-02-22 10:28AM'}


DENGBR12.parquet: 100%|██████████| 1.12M/1.12M [00:27<00:00, 40.5kB/s]


{'name': 'DENGBR13.dbc', 'disease': 'Dengue', 'year': 2013, 'size': '65.5 MB', 'last_update': '2022-02-18 10:03AM'}


DENGBR13.parquet: 100%|██████████| 2.40M/2.40M [00:59<00:00, 40.6kB/s]


{'name': 'DENGBR14.dbc', 'disease': 'Dengue', 'year': 2014, 'size': '42.7 MB', 'last_update': '2021-11-23 12:15PM'}


...

DENGBR20.parquet: 100%|██████████| 2.40M/2.40M [01:16<00:00, 31.3kB/s]


{'name': 'DENGBR21.dbc', 'disease': 'Dengue', 'year': 2021, 'size': '42.9 MB', 'last_update': '2023-05-11 11:10AM'}


DENGBR21.parquet: 100%|██████████| 1.65M/1.65M [00:52<00:00, 31.4kB/s]


{'name': 'DENGBR22.dbc', 'disease': 'Dengue', 'year': 2022, 'size': '62.2 MB', 'last_update': '2023-08-21 03:20PM'}


DENGBR22.parquet: 100%|██████████| 2.27M/2.27M [01:12<00:00, 31.4kB/s]


#### SINASC

In [89]:
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
sinasc = SINASC().load()
sinasc_files = sinasc.get_files(group='DN', uf=states, year=years)

parquet_path = os.getcwd() + "/Data/Parquet/"
csv_path = os.getcwd() + '/Data/CSV/'

if not 'Parquet' in os.listdir(os.getcwd() + "/Data") :
      os.mkdir(parquet_path)

if not 'CSV' in os.listdir(os.getcwd() + "/Data") :
      os.mkdir(csv_path)

for file in sinasc_files:
      if not file.basename.replace('.dbc', '.parquet') in os.listdir(parquet_path) and not file.basename.replace('.dbc', '.csv') in os.listdir(csv_path):
            print(sinasc.describe(file))
            sinasc.download(file, local_dir=parquet_path)

{'name': 'DNAC2012.DBC', 'group': 'Declarações de Nascidos Vivos', 'uf': 'Acre', 'year': 2012, 'size': '1.0 MB', 'last_update': '2020-01-27 12:05PM'}


DNAC2012.parquet: 100%|██████████| 40.7k/40.7k [00:00<00:00, 84.1kB/s]


{'name': 'DNAC2013.dbc', 'group': 'Declarações de Nascidos Vivos', 'uf': 'Acre', 'year': 2013, 'size': '992.4 kB', 'last_update': '2020-01-27 12:05PM'}


DNAC2013.parquet: 100%|██████████| 44.5k/44.5k [00:00<00:00, 86.8kB/s]


{'name': 'DNAC2014.dbc', 'group': 'Declarações de Nascidos Vivos', 'uf': 'Acre', 'year': 2014, 'size': '874.8 kB', 'last_update': '2018-04-20 05:04PM'}


...

DNTO2020.parquet: 100%|██████████| 26.6k/26.6k [00:00<00:00, 35.1kB/s]


{'name': 'DNTO2021.dbc', 'group': 'Declarações de Nascidos Vivos', 'uf': 'Tocantins', 'year': 2021, 'size': '1.2 MB', 'last_update': '2023-04-28 12:53PM'}


DNTO2021.parquet: 100%|██████████| 26.6k/26.6k [00:00<00:00, 38.0kB/s]


{'name': 'DNTO2022.dbc', 'group': 'Declarações de Nascidos Vivos', 'uf': 'Tocantins', 'year': 2022, 'size': '1.1 MB', 'last_update': '2023-12-20 04:45PM'}


DNTO2022.parquet: 100%|██████████| 25.3k/25.3k [00:00<00:00, 40.3kB/s]


#### SIM

In [90]:
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
sim = SIM().load()
sim_files = sim.get_files('CID10', uf=states, year=years)

parquet_path = os.getcwd() + "/Data/Parquet/"
csv_path = os.getcwd() + '/Data/CSV/'

if not 'Parquet' in os.listdir(os.getcwd() + "/Data") :
      os.mkdir(parquet_path)

if not 'CSV' in os.listdir(os.getcwd() + "/Data") :
      os.mkdir(csv_path)

for file in sim_files:
      if not file.basename.replace('.dbc', '.parquet') in os.listdir(parquet_path) and not file.basename.replace('.dbc', '.csv') in os.listdir(csv_path):
            print(sim.describe(file))
            sim.download(file, local_dir=parquet_path)

{'name': 'DOAC2012.DBC', 'uf': 'Acre', 'year': 2012, 'group': 'CID10', 'size': '253.3 kB', 'last_update': '2020-01-31 02:48PM'}


DOAC2012.parquet: 100%|██████████| 11.0k/11.0k [00:00<00:00, 81.2kB/s]


{'name': 'DOAC2013.dbc', 'uf': 'Acre', 'year': 2013, 'group': 'CID10', 'size': '237.6 kB', 'last_update': '2020-01-31 02:48PM'}


/home/vicente/.local/lib/python3.10/site-packages/tqdm/std.py:533: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
DOAC2013.parquet: 100%|██████████| 10.3k/10.3k [00:00<00:00, 80.1kB/s]


{'name': 'DOAC2014.dbc', 'uf': 'Acre', 'year': 2014, 'group': 'CID10', 'size': '349.0 kB', 'last_update': '2020-01-31 02:48PM'}


...

DOTO2020.parquet: 100%|██████████| 22.7k/22.7k [00:00<00:00, 56.3kB/s]


{'name': 'DOTO2021.dbc', 'uf': 'Tocantins', 'year': 2021, 'group': 'CID10', 'size': '1.0 MB', 'last_update': '2023-04-28 11:56AM'}


DOTO2021.parquet: 100%|██████████| 28.2k/28.2k [00:00<00:00, 57.6kB/s]


{'name': 'DOTO2022.dbc', 'uf': 'Tocantins', 'year': 2022, 'group': 'CID10', 'size': '827.7 kB', 'last_update': '2023-12-21 04:20PM'}


DOTO2022.parquet: 100%|██████████| 23.0k/23.0k [00:00<00:00, 58.0kB/s]


#### Transformação Parquet para CSV

Transforma todos os .parquet em .csv

In [92]:
error_log_path = os.getcwd() + '/Data/error_log.txt'

files_names = os.listdir(parquet_path)
error_files = []

for file in files_names:
    try:
        df = pd.read_parquet(path= parquet_path + file)
        print(file)
        df.to_csv(path_or_buf=(csv_path + file.replace('.parquet', '.csv')), sep=';')
        
        parquet_files = os.listdir(parquet_path + file)
        for pFile in parquet_files:
            os.remove(parquet_path + file + '/' + pFile)
        os.rmdir(parquet_path + file) 
    except Exception as e:
        error_files.append(file)
        print(f"Erro ao processar o arquivo: {file}")

        os.remove(path=parquet_path + file)



# Escrever os arquivos que causaram erro em um arquivo de texto
if len(error_files) > 0:
    with open(error_log_path, 'w') as f:
        for error_file in error_files:
            f.write(f"{error_file}\n")